<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/49_5_Speech_Technologies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка библиотек
%%capture
!pip install SpeechRecognition pydub
!apt install ffmpeg

In [2]:
# Импорт библиотек
import os
import speech_recognition as sr
from pydub import AudioSegment
from pydub.generators import WhiteNoise
import numpy as np
from sklearn.metrics import accuracy_score

In [12]:
# Функция конвертации mp3 в wav
def convert_mp3_to_wav(input_mp3_path, output_wav_path=None, sample_rate=16000):

    # Загружаем MP3 файл
    audio = AudioSegment.from_mp3(input_mp3_path)

    # Устанавливаем частоту дискретизации
    audio = audio.set_frame_rate(sample_rate)

    # Экспортируем в WAV
    audio.export(output_wav_path, format="wav")
    print(f"Файл успешно конвертирован: {output_wav_path}")

    return output_wav_path

In [ ]:
# Функция добавления шума к записям
def add_noise_to_audio(input_path, output_path=None, noise_level=0.1):

    # Загружаем аудиофайл
    audio = AudioSegment.from_file(input_path)

    # Генерируем белый шум той же длительности, что и аудио
    noise = WhiteNoise().to_audio_segment(duration=len(audio))

    # Накладываем шум на оригинальное аудио
    noisy_audio = audio.overlay(noise, gain_during_overlay=noise_level*20)

    # Сохраняем результат
    noisy_audio.export(output_path, format=output_path.split('.')[-1])

    print(f"Файл с шумом успешно создан: {output_path}")
    return output_path

In [14]:
# Конвертирум mp3 в wav
folder_path = '/content/'

# Получаем список всех файлов и папок
all_items = os.listdir(folder_path)

# Фильтруем только файлы (исключаем подпапки)
files = [item for item in all_items if os.path.isfile(os.path.join(folder_path, item))]

i = 1
for file in files:
    convert_mp3_to_wav(file, f"clean/clean_sample{i}.wav")
    i += 1

Файл успешно конвертирован: clean/clean_sample1.wav
Файл успешно конвертирован: clean/clean_sample2.wav
Файл успешно конвертирован: clean/clean_sample3.wav
Файл успешно конвертирован: clean/clean_sample4.wav
Файл успешно конвертирован: clean/clean_sample5.wav


In [ ]:
# Добавляем шум к записям
add_noise_to_audio("sample3.wav", "noisy_samples/loud_noise.wav")